In [ ]:
import pandas as pd
import altair as alt
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn import metrics
from sklearn.tree import DecisionTreeClassifier

from sklearn.ensemble import RandomForestClassifier



alt.data_transformers.enable('json')

In [ ]:
denver = pd.read_csv('https://github.com/byuidatascience/data4dwellings/raw/master/data-raw/dwellings_denver/dwellings_denver.csv')

dwellings_ml = pd.read_csv("https://github.com/byuidatascience/data4dwellings/raw/master/data-raw/dwellings_ml/dwellings_ml.csv")

neighborhood = pd.read_csv("https://github.com/byuidatascience/data4dwellings/raw/master/data-raw/dwellings_neighborhoods_ml/dwellings_neighborhoods_ml.csv")


Create 2-3 charts that evaluate potential relationships between the house variables and the variable before1980 Explain what you learn from the charts that could help a machine learning algorithm.

In [ ]:
#create T/F column for before 1980
denver['before1980'] = denver.yrbuilt < 1980

avgdata = denver.groupby('before1980').agg(avg_netprice = ('netprice', np.mean)).reset_index()
denver

In [ ]:
netprice = (alt.Chart(denver)
    .mark_circle()
    .encode(
        alt.X('yrbuilt', scale = alt.Scale(zero = False),
        axis =alt.Axis(format = 'd')),
        alt.Y('netprice', axis = alt.Axis(title= 'Net Price ($)')))
        .properties(
        height = 150,
        width = 400,
        title = {'text': "Net Prices relation with Year Built"})
)

netprice.save('netprice.png')

This plot seems to be a jumbled mess. It is hard to tell if there is any real telling details between the net price of a home and the year it was built. Of note, there were many more homes after 1980 with drastically high net prices.

In [ ]:
livearea = (alt.Chart(denver)
    .mark_circle()
    .encode(
        alt.X('yrbuilt', scale = alt.Scale(zero = False),
        axis =alt.Axis(format = 'd')),
        alt.Y('livearea', axis = alt.Axis(title= 'Liveable Area (ft.)')))
        .properties(
        height = 150,
        width = 400,
        title = {'text': "Livable Areas relation with Year Built"})
)

livearea.save('livearea.png')

Build a classification model labeling houses as being built “in or before 1980” or “after 1980”. Your goal is to reach 90% accuracy. Explain your final model choice (algorithm, tuning parameters, etc) and describe what other models you tried.

In [ ]:
bob1 = train_test_split(dwellings_ml)

# Organize Data for Machine Learning 

In [ ]:
targets = dwellings_ml.before1980
features = dwellings_ml.drop(columns=['before1980', 'yrbuilt', 'parcel'])
#features = dwellings_ml.filter(['nocars', 'numbaths'])
#features = dwellings_ml.filter(['livearea', 'basement', 'stories', 'nocars', 'numbaths'])

features_train, features_test, targets_train, targets_test = train_test_split(
    features, 
    targets, 
    test_size = .3, 
    random_state = 24601) 

In [55]:
classifierGNB = GaussianNB()

classifierGNB.fit(features_train, targets_train)

targets_predicted = classifierGNB.predict(features_test)

targets_test
targets_predicted
metrics.accuracy_score(targets_test, targets_predicted)

0.664969450101833

In [56]:
classifierDT = DecisionTreeClassifier(max_depth=10)

classifierDT.fit(features_train, targets_train)

targets_predicted = classifierDT.predict(features_test)

metrics.accuracy_score(targets_test, targets_predicted)

0.8945301134710504

In [ ]:
classifierRF = RandomForestClassifier(n_estimators=10)

classifierRF.fit(features_train, targets_train)

targets_predicted = classifierRF.predict(features_test)

metrics.accuracy_score(targets_test, targets_predicted)

Question 2

MY random model was a Random Forest with 10 estimators. Reached 92% accuracy. My targets was the dwelling denver data with 'before1980', 'yrbuilt', 'parcel' columns

Justify your classification model by discussing the most important features selected by your model. This discussion should include a chart and a description of the features.

In [ ]:
classifierRF.feature_importances_

In [ ]:
feature_df = pd.DataFrame({'features':features.columns, 'importance':classifierRF.feature_importances_})
top10 = feature_df[feature_df['importance'] >= 0.045].sort_values('importance', ascending=False)



### plot of features and importance

In [ ]:
top10features = (alt.Chart(top10)
.mark_bar()
.encode(x = 'importance',
        y = 'features')
)
top10features.save('top10.png')

Describe the quality of your classification model using 2-3 different evaluation metrics. You also need to explain how to interpret each of the evaluation metrics you use.

In [ ]:
metrics.plot_confusion_matrix(classifierRF, features_test, targets_test)

In [ ]:
(2334 +3968) /6874

recall = 3968/(3968+325)
recall #0.92429

precision = 3968/(3968+247)
precision #0.94139

In [ ]:
metrics.balanced_accuracy_score(targets_test, targets_predicted)

In [ ]:
print(metrics.classification_report(targets_test, targets_predicted))